In [472]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge


from sklearn.impute import SimpleImputer
np.random.seed(123)

df_train = pd.read_csv('train.csv')
df_train.head()
df_train.set_index('Id', inplace=True)
print(df_train.shape)
df_train.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [473]:
all_null_values = df_train.isnull().sum()
g50_null = all_null_values[all_null_values>50]
print(g50_null)
df_train.drop(g50_null.index, axis=1,inplace=True)
df_train.shape

LotFrontage      259
Alley           1369
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


(1460, 69)

In [474]:
X = df_train.drop('SalePrice', axis=1)
y = df_train['SalePrice']
categorical_features = df_train.select_dtypes(include='object').columns.tolist()
numerical_features = df_train.drop('SalePrice', axis=1).select_dtypes(exclude='object').columns.tolist()

fill_mising_transformer = ColumnTransformer([('impuer_cat', SimpleImputer(strategy='most_frequent'), categorical_features), ('imputer_num', SimpleImputer(strategy='mean'), numerical_features)])
X_filled = fill_mising_transformer.fit_transform(X)

ohe = OneHotEncoder(drop='first')
X_cat_ohe = ohe.fit_transform(X_filled[:, :len(categorical_features)]).toarray()
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_filled[:, len(categorical_features):])
X = np.concatenate((np.array(X_cat_ohe), np.array(X_num_scaled)), axis=1)
X = pd.DataFrame(X, index=df_train.index)
assert X.notna().all().all()
print(X.shape)
X.head() 



(1460, 215)


,0,1,2,3,4,5,6,7,8,9,...,205,206,207,208,209,210,211,212,213,214
Id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
5,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777


In [479]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3, random_state=42)

clf = Ridge(alpha=1)
clf.fit(X_train, y_train)
clf.score(X_cv, y_cv)

0.8757831163925499

In [481]:
clf.score(X_train, y_train)

0.905869625784984

In [482]:
df_test = pd.read_csv('test.csv')
df_test.set_index('Id', inplace=True)
df_test.drop(g50_null.index, axis=1,inplace=True)
assert np.array(df_train.drop('SalePrice', axis=1).columns==df_test.columns).all()

X_test_filled = fill_mising_transformer.transform(df_test)

X_test_cat_ohe = ohe.transform(X_test_filled[:, :len(categorical_features)]).toarray()

X_test_num_scaled = scaler.transform(X_test_filled[:, len(categorical_features):])
X_test = np.concatenate((np.array(X_test_cat_ohe), np.array(X_test_num_scaled)), axis=1)
X_test = pd.DataFrame(X_test, index=df_test.index)
assert X_test.notna().all().all()

df_test['SalePrice'] = clf.predict(X_test)
df_test['SalePrice'].to_csv('output.csv', header=True)